## Deploy Text Generation Model (Cohere Generate - Command Medium)

##### Prerequisites 

In [ ]:
%%capture

!pip install cohere-sagemaker

#### Imports 

In [2]:
from sagemaker import get_execution_role
from cohere_sagemaker import CohereError
from cohere_sagemaker import Client
from sagemaker import ModelPackage
import numpy as np
import sagemaker
import logging
import boto3
import time

##### Setup logging 

In [3]:
logger = logging.getLogger('sagemaker')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

##### Log versions of dependencies 

In [4]:
logger.info(f'Using sagemaker=={sagemaker.__version__}')
logger.info(f'Using boto3=={boto3.__version__}')

Using sagemaker==2.140.1
Using boto3==1.26.96


#### Setup essentials

Mapping for Model Packages (initially only us-east-1 and eu-west-1 is supported)

In [5]:
model_package_map = {
    'us-east-1': 'arn:aws:sagemaker:us-east-1:865070037744:model-package/cohere-gpt-medium-v1-4-825b877abfd53d7ca65fd7b4b262c421',
    'eu-west-1': 'arn:aws:sagemaker:eu-west-1:985815980388:model-package/cohere-gpt-medium-v1-4-825b877abfd53d7ca65fd7b4b262c421'
}

In [6]:
region = boto3.Session().region_name
logger.info(f'Region = {region}')

Region = us-east-1


In [7]:
if region not in model_package_map.keys():
    raise Exception(f'Unsupported region = {region}')

In [8]:
MODEL_PACKAGE_ARN = model_package_map[region]
logger.info(f'Model package ARN = {MODEL_PACKAGE_ARN}')

Model package ARN = arn:aws:sagemaker:us-east-1:865070037744:model-package/cohere-gpt-medium-v1-4-825b877abfd53d7ca65fd7b4b262c421


In [9]:
ROLE = get_execution_role()
session = sagemaker.Session()
logger.info(f'Role = {ROLE}')

Role = arn:aws:iam::119174016168:role/service-role/AmazonSageMaker-ExecutionRole-20211014T093628


In [10]:
timestamp = int(time.time())
MODEL_NAME = f'cohere-medium-{timestamp}'

#### Create a SageMaker model

In [11]:
model = ModelPackage(role=ROLE, 
                     model_package_arn=MODEL_PACKAGE_ARN, 
                     sagemaker_session=session, 
                     name=MODEL_NAME)
model.__dict__

{'model_data': None,
 'image_uri': None,
 'role': 'arn:aws:iam::119174016168:role/service-role/AmazonSageMaker-ExecutionRole-20211014T093628',
 'predictor_cls': None,
 'env': {},
 'name': 'cohere-medium-1680827379',
 '_base_name': None,
 'vpc_config': None,
 'sagemaker_session': <sagemaker.session.Session at 0x7fa55e0bcb50>,
 'endpoint_name': None,
 '_is_compiled_model': False,
 '_compilation_job_name': None,
 '_is_edge_packaged_model': False,
 'inference_recommender_job_results': None,
 'inference_recommendations': None,
 '_enable_network_isolation': False,
 'model_kms_key': None,
 'image_config': None,
 'entry_point': None,
 'source_dir': None,
 'dependencies': [],
 'git_config': None,
 'container_log_level': 20,
 'bucket': None,
 'key_prefix': None,
 'uploaded_code': None,
 'repacked_model_data': None,
 'algorithm_arn': None,
 'model_package_arn': 'arn:aws:sagemaker:us-east-1:865070037744:model-package/cohere-gpt-medium-v1-4-825b877abfd53d7ca65fd7b4b262c421',
 '_created_model_packag

#### Deploy the text generation model as a SageMaker endpoint for real-time synchronous inference

In [12]:
NUM_INSTANCES = 1
INSTANCE_TYPE = 'ml.g5.xlarge'

In [13]:
%%time

model.deploy(NUM_INSTANCES, 
             INSTANCE_TYPE, 
             endpoint_name=MODEL_NAME)

Creating model with name: cohere-medium-1680827379
CreateModel request: {
    "ModelName": "cohere-medium-1680827379",
    "ExecutionRoleArn": "arn:aws:iam::119174016168:role/service-role/AmazonSageMaker-ExecutionRole-20211014T093628",
    "Containers": [
        {
            "ModelPackageName": "arn:aws:sagemaker:us-east-1:865070037744:model-package/cohere-gpt-medium-v1-4-825b877abfd53d7ca65fd7b4b262c421"
        }
    ],
    "EnableNetworkIsolation": true
}
Creating endpoint-config with name cohere-medium-1680827379
Creating endpoint with name cohere-medium-1680827379


-----------!CPU times: user 180 ms, sys: 25.2 ms, total: 206 ms
Wall time: 6min 3s
